# Detailed Charge Analysis: Bookings & Releases

This notebook loads detailed booking and release records to allow interactive searching of charges with date and individual details.

## 1. Setup and Data Loading

In [1]:
import pandas as pd
import glob
import ipywidgets as widgets
from IPython.display import display, clear_output

bookings_list = []
releases_list = []

# 1. Load Bookings
booking_files = sorted(glob.glob('JABookings_*.csv'))
print(f"Processing {len(booking_files)} Booking files...")
for f in booking_files:
    try:
        # Read specific columns
        df = pd.read_csv(f, usecols=['ConfineDate', 'InmateFullName', 'ChargeOffenseDescription'], dtype=str)
        # Standardize column names
        df = df.rename(columns={
            'ConfineDate': 'Date',
            'InmateFullName': 'Name',
            'ChargeOffenseDescription': 'Charge'
        })
        df['Type'] = 'Booking'
        bookings_list.append(df)
    except Exception as e:
        # print(f"Error reading {f}: {e}") # Optional: uncomment to debug
        pass

# 2. Load Releases
release_files = sorted(glob.glob('JAReleases_*.csv'))
print(f"Processing {len(release_files)} Release files...")
for f in release_files:
    try:
        # Read specific columns (Note: InmateName vs InmateFullName)
        df = pd.read_csv(f, usecols=['ReleaseDate', 'InmateName', 'OffenseDescription'], dtype=str)
        # Standardize column names
        df = df.rename(columns={
            'ReleaseDate': 'Date',
            'InmateName': 'Name',
            'OffenseDescription': 'Charge'
        })
        df['Type'] = 'Release'
        releases_list.append(df)
    except Exception as e:
        # print(f"Error reading {f}: {e}")
        pass

# 3. Combine Data
if bookings_list or releases_list:
    all_data = pd.concat(bookings_list + releases_list, ignore_index=True)
    
    # Create a helper column for sorting by date
    all_data['Date_Sort'] = pd.to_datetime(all_data['Date'], errors='coerce')
    all_data = all_data.sort_values(by='Date_Sort', ascending=False)
    
    # Drop helper and fill NaNs
    all_data = all_data.drop(columns=['Date_Sort']).fillna('')
    
    # Reorder columns for display
    all_data = all_data[['Date', 'Type', 'Name', 'Charge']]
    
    print(f"Successfully loaded {len(all_data)} total records.")
    # print("Sample data:")
    # display(all_data.head())
else:
    all_data = pd.DataFrame(columns=['Date', 'Type', 'Name', 'Charge'])
    print("No data found.")

Processing 30 Booking files...
Processing 30 Release files...
Successfully loaded 11368 total records.


## 2. Charge Summary
Below is the full list of charges sorted by frequency (Count).

In [4]:
# Group by Charge and count occurrences

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

if not all_data.empty:
    charge_summary = all_data['Charge'].value_counts().reset_index()
    charge_summary.columns = ['Charge', 'Count']
    
    print(f"Total unique charges: {len(charge_summary)}")
    display(charge_summary)
else:
    print("No data available for summary.")

Total unique charges: 635


,Charge,Count
0,ASSAULT CAUSES BODILY INJURY FAMILY MEMBER,900
1,DRIVING WHILE INTOXICATED,732
2,POSS CS PG 1/1-B <1G,556
3,CRIMINAL TRESPASS,458
4,POSS CS PG 1/1-B >=1G<4G,390
5,POSS MARIJ < 2OZ,354
6,UNL CARRYING WEAPON,289
7,EVADING ARREST DETENTION,260
8,AGG ASSLT W/DEADLY WEAPON,255
9,DRIVING WHILE INTOXICATED BAC >= 0.15,238


## 3. Interactive Search
Search for a charge to see all associated individuals and dates.

In [3]:
# Setup Widgets
text_input = widgets.Text(
    value='',
    placeholder='Type charge name (e.g., THEFT, ASSAULT)...',
    description='Search Charge:',
    layout=widgets.Layout(width='50%')
)

output = widgets.Output()

def on_text_change(change):
    with output:
        clear_output(wait=True)
        search_term = change['new'].strip().upper()
        
        if len(search_term) < 2:
            print("Please enter at least 2 characters to search.")
            return
            
        # Filter data
        # Case-insensitive partial match
        mask = all_data['Charge'].str.contains(search_term, case=False, na=False)
        results = all_data[mask]
        
        if results.empty:
            print(f"No records found for '{search_term}'.")
        else:
            print(f"Found {len(results)} records matching '{search_term}':")
            # Display full table, or limit if too large (e.g., top 100)
            if len(results) > 100:
                print("(Showing first 100 results)")
                display(results.head(100))
            else:
                display(results)

text_input.observe(on_text_change, names='value')

display(text_input, output)

# Initial state
with output:
    print("Enter a charge above to view details.")

Text(value='', description='Search Charge:', layout=Layout(width='50%'), placeholder='Type charge name (e.g., …

Output()